## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-23-18-52-58 +0000,bbc,Search for new grooming inquiry head will 'tak...,https://www.bbc.com/news/articles/cvgwnqeq5z0o...
1,2025-10-23-18-51-55 +0000,startribune,Sophia Forchas leaves hospital to boisterous w...,https://www.startribune.com/sophia-forchas-lea...
2,2025-10-23-18-51-46 +0000,startribune,"Brooks: In Minneapolis, empty space downtown m...",https://www.startribune.com/in-minneapolis-emp...
3,2025-10-23-18-51-09 +0000,nyt,Trump Officials Shoot Down Israeli Aspirations...,https://www.nytimes.com/2025/10/23/world/middl...
4,2025-10-23-18-50-59 +0000,nyt,A Costly Radio System Faltered When Texas Need...,https://www.nytimes.com/2025/10/23/us/texas-fl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,59
1,new,25
227,russian,15
103,sanctions,14
41,off,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
363,2025-10-22-20-58-00 +0000,wsj,President Trump has privately told allies he b...,https://www.wsj.com/politics/elections/trump-z...,146
280,2025-10-23-03-10-21 +0000,nypost,Entire White House East Wing will be demolishe...,https://nypost.com/2025/10/22/us-news/entire-w...,138
183,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...,128
158,2025-10-23-13-32-00 +0000,nyt,Trump Imposes Sanctions on Russian Oil Compani...,https://www.nytimes.com/2025/10/22/us/politics...,119
308,2025-10-23-00-34-00 +0000,wsj,The White House is tearing down the entire Eas...,https://www.wsj.com/politics/policy/white-hous...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
363,146,2025-10-22-20-58-00 +0000,wsj,President Trump has privately told allies he b...,https://www.wsj.com/politics/elections/trump-z...
327,67,2025-10-22-22-47-41 +0000,wapo,U.S. imposes sanctions on Russian oil companie...,https://www.washingtonpost.com/world/2025/10/2...
22,60,2025-10-23-18-21-00 +0000,wsj,Top Republicans warned that air-traffic contro...,https://www.wsj.com/politics/policy/air-traffi...
280,54,2025-10-23-03-10-21 +0000,nypost,Entire White House East Wing will be demolishe...,https://nypost.com/2025/10/22/us-news/entire-w...
340,49,2025-10-22-22-08-46 +0000,nypost,Elon Musk’s Tesla disappoints investors despit...,https://nypost.com/2025/10/22/business/elon-mu...
284,48,2025-10-23-02-41-32 +0000,nypost,Mom of 4 shot dead by ‘obsessed’ ex-boyfriend ...,https://nypost.com/2025/10/22/world-news/young...
381,46,2025-10-22-20-16-45 +0000,nypost,GM embraces AI with plans to use ‘eyes-off’ dr...,https://nypost.com/2025/10/22/business/gm-embr...
299,42,2025-10-23-01-23-33 +0000,nypost,Andrew Cuomo bolts for Knicks opener minutes a...,https://nypost.com/2025/10/22/us-news/andrew-c...
187,41,2025-10-23-11-35-08 +0000,cbc,King Charles becomes the first British royal t...,https://www.cbc.ca/news/world/rome-vatican-leo...
297,40,2025-10-23-01-32-10 +0000,nyt,Federal Agents Head to San Francisco Bay Area ...,https://www.nytimes.com/2025/10/22/us/border-p...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
